<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter

Начинаем с импорта библиотек

In [7]:
import pandas as pd
import psycopg2
import yaml
import sqlalchemy 

Создаем соединение с заданными параметрами

In [9]:
connection = psycopg2.connect(
    dbname=dbname,
    user=user,
    host=host,
    password=password,
    port=port
)

### Предварительный анализ данных

3.1 Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies).

In [11]:
# текст запроса
query_3_1 = f'''SELECT COUNT(v.id) AS "Кол-во вакансий"
                FROM public.vacancies v

            '''
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
df = pd.read_sql_query(query_3_1, connection)
df

,Кол-во вакансий
0,49197


3.2 Напишите запрос, который посчитает количество работодателей (таблица employers).

In [12]:
query_3_2 = f'''
    SELECT COUNT(name) AS "Кол-во работодателей"
    FROM public.employers
    '''

df = pd.read_sql_query(query_3_2, connection)
df

,Кол-во работодателей
0,23501


3.3 Напишите запрос, который посчитает количество регионов (таблица areas).

In [13]:
query_3_3 = f'''
    SELECT COUNT(name) AS "Кол-во регионов"
    FROM public.areas
    '''
    
df = pd.read_sql_query(query_3_3, connection)
df

,Кол-во регионов
0,1362


3.4 Напишите запрос, который посчитает количество сфер деятельности (таблица industries).

In [14]:
query_3_4 = f'''
    SELECT COUNT(name) AS "Кол-во сфер деятельности"
    FROM public.industries
    '''
    
df = pd.read_sql_query(query_3_4, connection)
df

,Кол-во сфер деятельности
0,294


#### Выводы по предварительному анализу данных:
- База данных содержит достаточно большое количество вакансий (49197). Это может указывать на активность рынка труда и наличие разнообразных возможностей для соискателей.
- Количество работодателей в базе составляет 23501. Это означает, что на рынке труда представлено большое количество компаний и организаций, которые предлагают вакансии.
- В базе данных содержится информация о 1362 регионах. Это указывает на географическое разнообразие вакансий и возможности трудоустройства в различных местах.
- Сферы деятельности представлены в базе данных 294. Это свидетельствует о разнообразии отраслей и секторов экономики, в которых существуют вакансии.

### Детальный анализ вакансий

4.1 Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).

In [15]:
query_4_1 = f'''
    SELECT
        COUNT(v.id) AS cnt,
        a.name AS area
    FROM public.vacancies v
    JOIN public.areas a ON a.id = v.area_id
    GROUP BY 2
    ORDER BY 1 DESC
    LIMIT 5
    '''
    
df = pd.read_sql_query(query_4_1, connection)
df

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


4.2 У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [16]:
query_4_2 = f'''
    SELECT COUNT(*) AS "Кол-во вакансий"
    FROM public.vacancies v
    WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
    '''
    
df = pd.read_sql_query(query_4_2, connection)
df

,Кол-во вакансий
0,24073


4.3 Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.

In [17]:
query_4_3 = f'''
    SELECT 
        ROUND(AVG(v.salary_from)) AS "Среднее значение для нижней зп вилки",
        ROUND(AVG(v.salary_to)) AS "Среднее значение для верхней зп вилки"
    FROM public.vacancies v
   
    '''
df = pd.read_sql_query(query_4_3, connection)
df

,Среднее значение для нижней зп вил,Среднее значение для верхней зп ви
0,71065.0,110537.0


Задание 4.4 Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Какая пара находится на втором месте по популярности?

In [18]:
query_4_4 = f'''
    SELECT
        COUNT(v.id) AS "Кол-во вакансий",
        v.schedule AS "Тип рабочего графика",
        v.employment AS "Тип трудоустройства"
    FROM public.vacancies v
    GROUP BY v.schedule, v.employment
    ORDER BY 1 DESC
    OFFSET 1
    LIMIT 1
    '''
df = pd.read_sql_query(query_4_4, connection)
df

,Кол-во вакансий,Тип рабочего графика,Тип трудоустройства
0,7802,Удаленная работа,Полная занятость


4.5 Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [37]:
query_4_4 = f'''
    SELECT 
        COUNT(v.id) AS cnt,
        v.experience AS "Требуемый опыт работы"
    FROM public.vacancies v
    GROUP BY v.experience
    ORDER BY 1
    '''
df = pd.read_sql_query(query_4_4, connection)
df

,cnt,Требуемый опыт работы
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


#### Выводы по детальному анализу вакансий:
- Пятёрка лидеров по количеству вакансий, включает Москву, Санкт-Петербург, Минск, Новосибирск и Алматы. Это города, в которых предложение вакансий наиболее высоко, что может указывать на их значимость и развитость как центров бизнеса и экономики.
- Информация о зарплате указана для 24073 вакансий. Это позволяет провести анализ зарплатного уровня и оценить рыночную стоимость работы.
- Среднее значение для нижней зарплатной вилки составляет 71065, а для верхней зарплатной вилки - 110537. Это диапазон зарплат, предлагаемых во многих вакансиях, и может служить ориентиром для соискателей в планировании своей заработной платы.
- Опыт работы от 1 года до 3 лет является наиболее востребованным. Это может указывать на то, что многие работодатели ищут кандидатов с определенным уровнем опыта, который считается оптимальным для выполнения задач в соответствующей области.
- Полная занятость является наиболее популярным типом занятости. Это означает, что большинство работодателей ищут сотрудников, готовых работать на полный рабочий день.

### Анализ работодателей

5.1 Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [20]:
query_5_1 = f'''
    (SELECT 
        e.name AS "Работодатель",
        COUNT(v.id) AS "Кол-во вакансий"
    FROM public.employers e
    JOIN public.vacancies v ON v.employer_id = e.id
    GROUP BY e.name
    ORDER BY 2 DESC
    LIMIT 1
    )
    
    UNION ALL
    
    (SELECT 
        e.name AS "Работодатель",
        COUNT(v.id) AS "Кол-во вакансий"
    FROM public.employers e
    JOIN public.vacancies v ON v.employer_id = e.id
    GROUP BY e.name
    ORDER BY 2 DESC
    OFFSET 4
    LIMIT 1
    --OFFSET 4
    )
    '''
df = pd.read_sql_query(query_5_1, connection)
df

,Работодатель,Кол-во вакансий
0,Яндекс,1933
1,Газпром нефть,331


5.2 Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [21]:
query_5_2 = f'''
    SELECT 
        a.name AS "Регион",
        COUNT(e.name) AS "Кол-во работодателей",
        COUNT(v.name) AS "Кол-во вакансий"
    FROM public.areas a
        LEFT JOIN public.employers e ON a.id = e.area
        LEFT JOIN public.vacancies v ON v.area_id = a.id
    WHERE v.area_id IS NULL
    GROUP BY 1
    ORDER BY 3 ASC, 2 DESC
    LIMIT 5
    '''
    
df = pd.read_sql_query(query_5_2, connection)
df

,Регион,Кол-во работодателей,Кол-во вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


5.3 Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.

Выберите максимальное значение из получившегося списка.

In [22]:
query_5_3 = f'''
    SELECT       
        e.name AS "Работодатель",
       
        COUNT(DISTINCT v.area_id) AS "Кол-во регионов"
    FROM public.vacancies v
    FULL JOIN public.employers e ON v.employer_id = e.id

    GROUP BY 1
    ORDER BY 2 DESC
    '''
    
df = pd.read_sql_query(query_5_3, connection)
df

,Работодатель,Кол-во регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


5.4 Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

Введите количество, которое вернул запрос, в поле ниже.

In [23]:
query_5_4 = f'''
    SELECT       
        COUNT(*) AS "Кол-во работодателей"
    FROM public.employers_industries ei
    RIGHT JOIN public.employers e ON ei.employer_id = e.id
    WHERE ei.industry_id IS NULL
    '''
    
df = pd.read_sql_query(query_5_4, connection)
df

,Кол-во работодателей
0,8419


5.5 Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

Введите в поле ниже название этой компании так же, как оно указано в результате запроса.

In [39]:
query_5_5 = f'''
    SELECT
        e.name AS "Названия компаний",
        COUNT(ei.industry_id) AS "Кол-во сфер деятельности"
    FROM public.employers e
    FULL JOIN public.employers_industries ei ON ei.employer_id = e.id
    GROUP BY e.name
    HAVING COUNT(ei.industry_id) = 4
    ORDER BY 1
    OFFSET 2
    LIMIT 1
    '''

df = pd.read_sql_query(query_5_5, connection)
df

,Названия компаний,Кол-во сфер деятельности
0,2ГИС,4


5.6 С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [25]:
query_5_6 = f'''
    SELECT
        COUNT(ei.employer_id) AS "Кол-во работодателей"
    FROM public.employers_industries ei
    LEFT JOIN public.industries i ON ei.industry_id = i.id
    WHERE i.name = 'Разработка программного обеспечения'
    '''
    
df = pd.read_sql_query(query_5_6, connection)
df

,Кол-во работодателей
0,3553


5.7 Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице. Должна получиться выборка такого вида (приведён пример результата для компании SberTech):

In [26]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
tables = pd.read_html(url)
city_df = tables[1]
city_list = list(city_df['Город'])
city_list[13] = 'Воронеж'
display(city_list)

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [27]:
query_5_7 = f'''
    SELECT
        a.name AS "Регионы-миллионники",
        COUNT(v.id) "Кол-во вакансий"
    FROM public.vacancies v
    LEFT JOIN public.employers e on v.employer_id = e.id
    LEFT JOIN public.areas a on v.area_id = a.id
    WHERE e.name = 'Яндекс' and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 
    'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
    GROUP BY a.name

    UNION ALL

    SELECT 
        'Total', 
        count(*)
    FROM public.vacancies v
    LEFT JOIN public.employers e on v.employer_id = e.id
    LEFT JOIN public.areas a on v.area_id = a.id
    WHERE e.name = 'Яндекс' and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 
    'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
    ORDER BY 2
    '''
    
df = pd.read_sql_query(query_5_7, connection)
df

,Регионы-миллионники,Кол-во вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


#### Выводы по анализу работодателей:
- Компания Яндекс занимает первое место по количеству вакансий. Это указывает на значительное присутствие и активность компании на рынке труда.
- В регионах, где нет вакансий, большинство работодателей находится в России. Это может быть связано с ориентацией этих работодателей на местный рынок труда.
- Выявлено 8419 работодателей, у которых не указана сфера деятельности. Это может указывать на неполную или некорректную заполненность данных в базе.
- Наиболее востребованный опыт работы - от 1 года до 3 лет. Это может быть связано с тем, что работодатели ищут кандидатов, которые имеют определенный опыт работы, но еще не являются слишком специализированными.
- Полная занятость является наиболее популярным типом занятости. Это указывает на то, что работодатели предпочитают нанимать сотрудников на полный рабочий день.
- Компания Яндекс представила 485 вакансий в 17 городах-миллионниках. Это говорит о широком географическом охвате и наличии возможностей для трудоустройства в разных городах.

###  Предметный анализ

 6.1 Сколько вакансий имеет отношение к данным?

In [28]:
query_6_1 = f'''
    SELECT
        COUNT(v.id) AS "Кол-во вакансий, связанных с DS"
    FROM public.vacancies v
    WHERE lower(v.name) LIKE '%данн%' OR lower(v.name) LIKE '%data%'
    '''

df =pd.read_sql_query(sqlalchemy.text(query_6_1), connection)
df

,"Кол-во вакансий, связанных с DS"
0,1771


6.2 Сколько есть подходящих вакансий для начинающего дата-сайентиста?
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

‘data scientist’;
‘data science’;
‘исследователь данных’;
‘ML’ (здесь не нужно брать вакансии по HTML);
‘machine learning’;
‘машинн%обучен%’.

Считаем вакансиями для специалистов уровня Junior следующие:

в названии есть слово “junior” или
требуемый опыт — «Нет опыта» или
тип трудоустройства — «Стажировка».

In [29]:
query_6_2 = f'''
    SELECT
        COUNT(v.id) AS "Кол-во вакансий junior DS"
    FROM public.vacancies v
    WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) like '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    AND 
    (lower(v.name) LIKE '%junior%'
    OR lower(v.experience) LIKE '%нет опыта%'
    OR lower(v.experience) LIKE '%стажировка%')
    '''

df =pd.read_sql_query(sqlalchemy.text(query_6_2), connection)
df

,Кол-во вакансий junior DS
0,51


6.3 Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?
В качестве ответа запишите количество, которое вернул запрос.

In [30]:
query_6_3 = f'''
    SELECT
        COUNT(v.id)
    FROM public.vacancies v
    WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) like '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    AND
    (v.key_skills LIKE '%SQL%'
    OR v.key_skills LIKE '%postgres%')
    '''

df =pd.read_sql_query(sqlalchemy.text(query_6_3), connection)
df

,count
0,201


6.4 С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.
В качестве ответа запишите количество, которое вернул запрос.

In [31]:
query_6_4 = f'''
    SELECT
        COUNT(v.id)
    FROM public.vacancies v
    WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) like '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    AND v.key_skills LIKE '%Python%'
    '''

df =pd.read_sql_query(sqlalchemy.text(query_6_4), connection)
df

,count
0,351


6.5 Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [32]:
query_6_5 = f'''
    select
        round(AVG(LENGTH(v.key_skills) -
        LENGTH(REPLACE(v.key_skills, chr(9), '')) + 1), 2) AS "Среднее кол-во ключевых скилов"
    FROM public.vacancies v
    WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) like '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    '''

df =pd.read_sql_query(sqlalchemy.text(query_6_5), connection)
df

,Среднее кол-во ключевых скилов
0,6.41


6.6 Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).
Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [33]:
query_6_6 = f'''
    select
        round(AVG(COALESCE((v.salary_to + v.salary_from)/2, v.salary_to, v.salary_from))) AS "Средняя зарплата дата-сайентиста с опытом работы от 3 до 6 лет"
    FROM public.vacancies v
    WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) like '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    AND (v.salary_to IS NOT NULL OR v.salary_from IS NOT NULL)
    AND v.experience = 'От 3 до 6 лет'
    '''

df =pd.read_sql_query(sqlalchemy.text(query_6_6), connection)
df

,Средняя зарплата дата-сайентиста
0,243115.0


#### Выводы по предметному анализу:
- Количество вакансий, связанных с работой с данными, составляет 1771. Это указывает на наличие значительного спроса на специалистов в области анализа данных и data science.
- Существует 51 подходящая вакансия для начинающего дата-сайентиста. Это означает, что существуют возможности для новичков в этой области найти работу и начать свою карьеру.
- Количество вакансий для дата-сайентистов, где SQL или postgre указаны в качестве ключевых навыков, составляет 201. Это указывает на важность знания SQL или postgre для работы в этой области и требования работодателей в этом отношении.
- Количество вакансий, где Python указывается в качестве ключевого навыка, составляет 351. Это говорит о том, что знание Python востребовано в работе дата-сайентиста и является одним из основных инструментов в этой области.
- Среднее количество ключевых навыков для дата-сайентиста составляет более 6. Это говорит о том, что в этой области требуются специалисты с широким спектром навыков и компетенций.
- Средняя зарплата для дата-сайентиста с опытом работы от 3 до 6 лет составляет 243115. Это указывает на то, что опытные специалисты в этой области могут рассчитывать на высокую заработную плату.

Дополнительные исследования

7.1 Проведём анализ количества вакансий по сферам деятельности. Выведем ТОП-3 сфер деятельности по количеству вакансий.

In [35]:
query_7_1 = f'''SELECT
    i.name AS "Сфера деятельности",
    COUNT(v.id) AS "Количество вакансий"
FROM industries i
LEFT JOIN employers_industries ei ON i.id = ei.industry_id
LEFT JOIN vacancies v ON ei.employer_id = v.employer_id
GROUP BY i.name
ORDER BY COUNT(v.id) DESC
LIMIT 3
'''

df =pd.read_sql_query(sqlalchemy.text(query_7_1), connection)
df

,Сфера деятельности,Количество вакансий
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413


7.2 Проведём анализ количества работодателей по сферам деятельности. Выведем ТОП-3 сфер деятельности по количеству работодателей.

In [36]:
query_7_2 = f'''SELECT
    i.name AS "Сфера деятельности",
    COUNT(DISTINCT ei.employer_id) AS "Количество работодателей"
FROM employers_industries ei
JOIN industries i ON ei.industry_id = i.id
GROUP BY i.name
ORDER BY COUNT(DISTINCT ei.employer_id) DESC
LIMIT 3
'''

df =pd.read_sql_query(sqlalchemy.text(query_7_2), connection)
df

,Сфера деятельности,Количество работодателей
0,Разработка программного обеспечения,3553
1,"Системная интеграция, автоматизации технологи...",2993
2,"Интернет-компания (поисковики, платежные систе...",1675


In [ ]:
#закрываем соединение после окончания работы
connection.close()

### Выводы по дополнительным исследованиям:
- Сфера деятельности "Разработка программного обеспечения" имеет наибольшее количество вакансий (12499) и наибольшее количество работодателей (3553). Это указывает на высокий спрос на специалистов в этой области и наличие множества компаний, занимающихся разработкой программного обеспечения.
- Сфера деятельности "Системная интеграция, автоматизации технологий" также имеет значительное количество вакансий (11034) и работодателей (2993). Это говорит о том, что автоматизация технологических процессов и системная интеграция являются востребованными и перспективными областями.
- Сфера деятельности "Интернет-компания (поисковики, платежные системы и другие)" имеет меньшее количество вакансий (6413) и работодателей (1675) по сравнению с предыдущими двумя сферами. Однако она все равно представляет собой значительную долю рынка труда и может предоставлять интересные возможности для трудоустройства.

## Общий вывод по проекту:
- Выборка содержит 49197 вакансий, представлен 23501 работодатель, содержится информация о 1362 регионах по 294 сферам деятельности.
- Лидерами по количеству вакансий являются мегаполисы, выделяя их значимость на рынке труда и развитость как центров бизнеса и экономики.
- Среднее значение для нижней зарплатной вилки составляет 71065, а для верхней зарплатной вилки - 110537.
- Опыт работы от 1 года до 3 лет является наиболее востребованным.
- Полная занятость является наиболее популярным типом занятости.
- Компания "Яндекс" занимает первое место по количеству вакансий - 1933.
- Полная занятость является наиболее популярным типом занятости.
- Количество вакансий, связанных с работой с данными, составляет 1771. Количество вакансий, где Python указывается в качестве ключевого навыка, составляет 351. Среднее количество ключевых навыков для дата-сайентиста составляет более 6, при этом соискатель может расчитывать в среднем на зарплату в 243115 руб.
- Сфера деятельности "Разработка программного обеспечения" имеет наибольшее количество вакансий - 12499 и наибольшее количество работодателей - 3553.